In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.linear_model import LassoCV,RidgeCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm

In [2]:
%matplotlib inline
plt.style.use("seaborn-darkgrid")
random_state = 17
np.random.seed(random_state)

In [3]:
train_X = pd.read_csv("train_X.csv", index_col=[0])
train_Y = pd.read_csv("train_Y.csv", index_col=[0])

In [4]:
test_X = pd.read_csv("test_X.csv", index_col=[0])
test_Y = pd.read_csv("test_Y.csv", index_col=[0])

In [5]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=random_state)

In [6]:
param_grid = [
              {},
              {},
              {'learning_rate': [0.1, 0.3], 'max_depth':[10, 30], 'n_estimators':[50, 100, 150]},
              {'n_estimators':[100, 200, 300]},
             ]
estimators = [
              LinearRegression(),
              DecisionTreeRegressor(random_state=random_state),
              XGBRegressor(gpu_id=0, objective='reg:squarederror'),
              RandomForestRegressor(random_state=random_state),
             ]
for i in tqdm(range(len(estimators))):
    print( "*" * 100)
    print("Evaluating Model", estimators[i])
    grid_clf = GridSearchCV(estimators[i], param_grid=param_grid[i], cv=5, verbose=10, n_jobs=12)
    grid_clf.fit(train_X.head(50_000), train_Y.head(50_000))
    print("*" * 100)
    print("Best Score:", grid_clf.best_score_)
    print("Best Params:", grid_clf.best_params_)
    print("\n\n", "*" * 100)
    

  0%|          | 0/4 [00:00<?, ?it/s]****************************************************************************************************
Evaluating Model LinearRegression()
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 25%|██▌       | 1/4 [00:00<00:01,  1.51it/s]****************************************************************************************************
Best Score: 0.6299032581885775
Best Params: {}


 ****************************************************************************************************
****************************************************************************************************
Evaluating Model DecisionTreeRegressor(random_state=17)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
 50%|█████     | 2/4 [00:02<00:02,  1.17s/it]****************************************************************************************************
Best Score: 0.7286160056037798
Best Params: {}


 **********************************************************

In [8]:
model = XGBRegressor(learning_rate= 0.1, max_depth= 10, n_estimators= 150, gpu_id=0, objective='reg:squarederror')
model.fit(train_X, train_Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=150, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
prediction = model.predict(test_X)
sklearn.metrics.r2_score(test_Y, prediction)

0.9302023815520439